# Nivel Intermedio de Python

## Un resumen de las características más avanzadas de Python

Esta sección asume que ya conoces los fundamentos.

1. Comprensiones (Comprehensions)
2. Generadores (Generators)
3. Subclases, Indicadores de Tipo (Type Hints), Pydantic
4. Decoradores (Decorators)
5. Docker (no es realmente Python, ¡pero lo usamos para ejecutar código de Python!)

In [1]:
# Primero, creemos algunas cosas:

fruits = ["Apples", "Bananas", "Pears"]

book1 = {"title": "Great Expectations", "author": "Charles Dickens"}
book2 = {"title": "Bleak House", "author": "Charles Dickens"}
book3 = {"title": "An Book By No Author"}
book4 = {"title": "Moby Dick", "author": "Herman Melville"}

books = [book1, book2, book3, book4]

# Parte 1: Comprensiones de listas y diccionarios

In [2]:
# Suficientemente simple para empezar

for fruit in fruits:
    print(fruit)

Apples
Bananas
Pears


In [3]:
# Hagamos una nueva versión de las frutas

fruits_shouted = []
for fruit in fruits:
    fruits_shouted.append(fruit.upper())

fruits_shouted

['APPLES', 'BANANAS', 'PEARS']

In [4]:
# Probablemente ya conozcas esto
# Existe una construcción agradable en Python llamada "comprensión de listas" que hace esto:

fruits_shouted2 = [fruit.upper() for fruit in fruits]
fruits_shouted2

['APPLES', 'BANANAS', 'PEARS']

In [5]:
# Pero puede que no sepas que también puedes hacer esto para crear diccionarios:

fruit_mapping = {fruit: fruit.upper() for fruit in fruits}
fruit_mapping

{'Apples': 'APPLES', 'Bananas': 'BANANAS', 'Pears': 'PEARS'}

In [6]:
# también puedes usar la declaración if para filtrar los resultados

fruits_with_longer_names_shouted = [fruit.upper() for fruit in fruits if len(fruit)>5]
fruits_with_longer_names_shouted

['APPLES', 'BANANAS']

In [7]:
fruit_mapping_unless_starts_with_a = {fruit: fruit.upper() for fruit in fruits if not fruit.startswith('A')}
fruit_mapping_unless_starts_with_a 

{'Bananas': 'BANANAS', 'Pears': 'PEARS'}

In [8]:
# Otra comprensión

[book['title'] for book in books]

['Great Expectations', 'Bleak House', 'An Book By No Author', 'Moby Dick']

In [9]:
# Este código fallará con un error porque uno de nuestros libros no tiene autor

[book['author'] for book in books]

KeyError: 'author'

In [10]:
# Pero esto funcionará, porque get() devuelve None

[book.get('author') for book in books]

['Charles Dickens', 'Charles Dickens', None, 'Herman Melville']

In [11]:
# Y esta variación filtrará los None

[book.get('author') for book in books if book.get('author')]

['Charles Dickens', 'Charles Dickens', 'Herman Melville']

In [12]:
# Y esta versión lo convertirá en un conjunto (set), eliminando duplicados

set([book.get('author') for book in books if book.get('author')])

{'Charles Dickens', 'Herman Melville'}

In [13]:
# Y finalmente, esta versión es aún mejor
# las llaves crean un conjunto (set), por lo que esto es una comprensión de conjuntos

{book.get('author') for book in books if book.get('author')}

{'Charles Dickens', 'Herman Melville'}

# Parte 2: Generadores

Usamos Generadores en el curso porque los modelos de IA pueden devolver resultados en forma de stream.

En Python, los generadores son una forma especial de crear iteradores, que son objetos que se pueden recorrer de uno en uno. Los generadores facilitan el trabajo con datos de uno en uno, especialmente cuando no quieres cargar todo en memoria a la vez.

## Conceptos clave
1. Los **generadores** permiten iterar sobre los datos sin crear una lista completa en memoria. En su lugar, producen valores de uno en uno a medida que los necesitas.

2. **La palabra clave ``yield``** es el núcleo de un generador. Cuando una función tiene yield en lugar de return, se convierte en una función generadora. Cada vez que se llama a yield, la función se detiene, guardando su estado, y devuelve un valor a quien la llamó.

3. **Evaluación perezosa:** Los generadores no calculan valores hasta que realmente los necesitas, lo que ahorra memoria.

**Escribir una función generadora**
He aquí un ejemplo para ilustrar cómo funciona una función generadora.

**Ejemplo 1: Función Generadora Simple**
Creemos una función generadora que produzca números del 1 al 5, de uno en uno.

In [14]:
def number_generator():
    for i in range(1, 6):
        yield i

Fíjate en la línea ``yield i`` - esto es lo que hace que number_generator() sea un generador. Cuando llamas a ``yield``, Python pausa la función y guarda el estado actual, e ``i`` es devuelto a quien lo llamó.

**Usando el Generador** 

Para usar el generador, puedes hacer un bucle sobre él o llamar a next() directamente.

In [16]:
# Crear un generador
gen = number_generator()

# Usa un bucle para iterar sobre el generador
for number in gen:
    print(number)


1
2
3
4
5


**Explicación de la salida**

1. Cuando se inicia el bucle ``for``, ``gen`` comienza en el primer valor (1).
2. Llama a ``yield`` y produce 1, luego hace una pausa.
3. Cada vez que el bucle continúa, el generador se reanuda desde donde lo dejó, produciendo 2, 3, 4, y finalmente 5. 
4. Después de producir 5, el generador termina.

**Usando ``next()`` Directamente**

Alternativamente, puedes llamar a ``next()`` en el generador para obtener valores uno a uno.

In [19]:
gen = number_generator()
print(next(gen))  # Output: 1
print(next(gen))  # Output: 2
# Sigue iterando hasta que se obtiene un error StopIteration
# print(next(gen))
# print(next(gen))
# print(next(gen))
# print(next(gen))

1
2


Cuando el generador se agota, al llamar de nuevo a next() lanza una excepción StopIteration.

**Ejemplo 2: Generador infinito**

Supongamos que desea un generador que siga produciendo números indefinidamente.

In [ ]:
def infinite_generator():
    i = 1
    while True:
        yield i
        i += 1

# Usando el generador infinito
gen = infinite_generator()
print(next(gen))  # Output: 1
print(next(gen))  # Output: 2
print(next(gen))  # Output: 3
print(next(gen))  # Output: 4
print(next(gen))  # Output: 5
print(next(gen))  # Output: 6
print(next(gen))  # Output: 7

1
2
3
4
5
6
7


Dado que este generador no se detiene por sí mismo, se denomina generador infinito. Puedes utilizarlo para casos en los que sólo quieras una parte de la secuencia, como la generación de datos bajo demanda.

**Ventajas de los generadores**

- **Eficiencia de memoria:** No almacenan todos los valores en memoria; generan cada valor sólo cuando es necesario.
- **Evaluación perezosa:** Ideal para grandes conjuntos de datos, ya que calculan los valores sólo cuando es necesario.
- **Simplifica el código:** Son una excelente forma de manejar secuencias que se calculan sobre la marcha sin necesidad de estructuras de datos complejas.

Los generadores son potentes para manejar conjuntos de datos grandes o potencialmente infinitos de forma eficiente y elegante en Python.

**Uso Avanzado de los Generadores**

In [23]:
# Primero define un generador; parece una función, pero tiene `yield` en lugar de `return`

import time

def come_up_with_fruit_names():
    for fruit in fruits:
        time.sleep(1) # pensando en una fruta
        yield fruit

In [24]:
# Luego úsalo

for fruit in come_up_with_fruit_names():
    print(fruit)

Apples
Bananas
Pears


In [25]:
# Aquí hay otro

def authors_generator():
    for book in books:
        if book.get("author"):
            yield book.get("author")

In [26]:
# Úsalo

for author in authors_generator():
    print(author)

Charles Dickens
Charles Dickens
Herman Melville


In [27]:
# Aquí está lo mismo escrito con una comprensión de listas

def authors_generator():
    for author in [book.get("author") for book in books if book.get("author")]:
        yield author

In [28]:
# Úsalo

for author in authors_generator():
    print(author)

Charles Dickens
Charles Dickens
Herman Melville


In [29]:
# Aquí hay un buen atajo
# Puedes usar "yield from" para ceder cada elemento de un iterable

def authors_generator():
    yield from [book.get("author") for book in books if book.get("author")]

In [30]:
# Úsalo

for author in authors_generator():
    print(author)

Charles Dickens
Charles Dickens
Herman Melville


In [31]:
# Y finalmente - podemos reemplazar la comprensión de listas por una comprensión de conjuntos

def unique_authors_generator():
    yield from {book.get("author") for book in books if book.get("author")}

In [32]:
# Úsalo

for author in unique_authors_generator():
    print(author)

Herman Melville
Charles Dickens


In [ ]:
# Y para divertirse un poco - ¡presiona el botón de detener en la barra de herramientas cuando te aburras!
# Es como si hubiéramos creado nuestro propio Modelo de Lenguaje Grande... aunque no particularmente grande...
# Intenta entender por qué imprime una letra a la vez, en lugar de una palabra a la vez. Si no estás seguro, intenta eliminar la palabra clave "from" en todo el código.

import random
import time

pronouns = ["I", "You", "We", "They"]
verbs = ["eat", "detest", "bathe in", "deny the existence of", "resent", "pontificate about", "juggle", "impersonate", "worship", "misplace", "conspire with", "philosophize about", "tap dance on", "dramatically renounce", "secretly collect"]
adjectives = ["turqoise", "smelly", "arrogant", "festering", "pleasing", "whimsical", "disheveled", "pretentious", "wobbly", "melodramatic", "pompous", "fluorescent", "bewildered", "suspicious", "overripe"]
nouns = ["turnips", "rodents", "eels", "walruses", "kumquats", "monocles", "spreadsheets", "bagpipes", "wombats", "accordions", "mustaches", "calculators", "jellyfish", "thermostats"]

def infinite_random_sentences():
    while True:
        yield from random.choice(pronouns)
        yield " "
        yield from random.choice(verbs)
        yield " "
        yield from random.choice(adjectives)
        yield " "
        yield from random.choice(nouns)
        yield ". "

for letter in infinite_random_sentences():
    print(letter, end="", flush=True)
    time.sleep(0.02)

## Ejercicio

Escribe algunas clases de Python para el ejemplo de los libros.

Escribe una clase `Book` con un título y un autor. Incluye un método `has_author()`.

Escribe una clase `BookShelf` con una lista de libros. Incluye un método generador `unique_authors()`.

# Parte 3: Subclases, Indicadores de Tipo (Type Hints), Pydantic

## Aspectos Avanzados de Clases en Python

Este apartado cubre conceptos avanzados en clases de Python para quienes ya conocen lo básico (como `__init__`, `self`, instancias y clases). Se abordarán:

- Pistas de tipo (type hints)
- Subclases
- Métodos de clase (`@classmethod`)
- Ejemplo completo integrando estos conceptos

### 1. Pistas de Tipo (Type Hints)
Las pistas de tipo en Python permiten especificar el tipo esperado de una variable o argumento. Son útiles para mejorar la legibilidad del código, ayudar a herramientas como editores o linters, y evitar errores comunes. No son obligatorias ni se aplican en tiempo de ejecución.

In [36]:
class Libro:
    titulo: str
    autor: str

    def __init__(self, titulo: str, autor: str):
        self.titulo = titulo
        self.autor = autor

**Uso avanzado con Optional y Union**

Puedes indicar tipos opcionales o múltiples posibles con `Optional` y `Union` del módulo `typing`.

In [37]:
from typing import Optional, Union

class Libro:
    titulo: str
    autor: Optional[str] = None  # Puede ser str o None
    anio_publicacion: Union[int, str] = 'Desconocido'  # Puede ser int o str 

### 2. Subclases
Las subclases permiten crear una clase que hereda atributos y métodos de otra. Esto permite reutilizar y extender código de forma eficiente.

In [38]:
class Libro:
    def __init__(self, titulo: str, autor: str):
        self.titulo = titulo
        self.autor = autor

    def descripcion(self) -> str:
        return f"'{self.titulo}' por {self.autor}"

class EBook(Libro):
    def __init__(self, titulo: str, autor: str, formato_archivo: str):
        super().__init__(titulo, autor)
        self.formato_archivo = formato_archivo

    def descripcion(self) -> str:
        return f"'{self.titulo}' por {self.autor} (formato: {self.formato_archivo})"

### 3. Métodos de Clase
Un método de clase se define con el decorador `@classmethod` y recibe `cls` como primer argumento, que representa la clase y no una instancia específica. Se usa comúnmente para constructores alternativos.

In [ ]:
from typing import List

class Libro:
    def __init__(self, titulo: str, autor: str):
        self.titulo = titulo
        self.autor = autor

    def descripcion(self) -> str:
        return f"'{self.titulo}' por {self.autor}"

    @classmethod
    def crear_varios(cls, lista_info: List[tuple]) -> List['Libro']:
        return [cls(titulo, autor) for titulo, autor in lista_info]

In [40]:
libros = Libro.crear_varios([
    ("1984", "George Orwell"),
    ("Un mundo feliz", "Aldous Huxley")
])

for libro in libros:
    print(libro.descripcion())

'1984' por George Orwell
'Un mundo feliz' por Aldous Huxley


### 4. Ejemplo Completo Integrado

Aquí combinamos herencia, métodos de clase y pistas de tipo en una sola solución.

In [41]:
class ItemBiblioteca:
    def __init__(self, titulo: str, anio: int):
        self.titulo = titulo
        self.anio = anio

    def obtener_info(self) -> str:
        return f"{self.titulo} ({self.anio})"

class Libro(ItemBiblioteca):
    def __init__(self, titulo: str, anio: int, autor: str):
        super().__init__(titulo, anio)
        self.autor = autor

    def obtener_info(self) -> str:
        return f"'{self.titulo}' por {self.autor} ({self.anio})"

    @classmethod
    def desde_cadena(cls, info: str) -> 'Libro':
        titulo, autor, anio = info.split(', ')
        return cls(titulo, int(anio), autor)

# Uso del constructor alternativo
libro = Libro.desde_cadena("El guardián entre el centeno, J.D. Salinger, 1951")
print(libro.obtener_info())

'El guardián entre el centeno' por J.D. Salinger (1951)


# Parte 4: Decoradores

Este apartado explora los **decoradores**, una de las características más potentes y elegantes de Python. Permiten modificar o extender el comportamiento de funciones o métodos de forma limpia y reutilizable.

## 1. Definiendo un Decorador: Entendiendo su propósito y uso

Un decorador es, en esencia, una **función que recibe otra función como argumento, le añade alguna funcionalidad y devuelve otra función**. Esta técnica permite envolver el código de una función existente con lógica adicional sin modificar su estructura interna.

La sintaxis `@` es una forma de "azúcar sintáctico" que simplifica la aplicación de un decorador a una función.

**Propósito y Uso:**
Los decoradores son una implementación del patrón de diseño estructural *Decorator* y son extremadamente útiles para:
- **Logging**: Registrar cuándo se ejecuta una función y con qué argumentos.
- **Medición de rendimiento**: Calcular el tiempo de ejecución de una función.
- **Control de acceso y permisos**: Verificar si un usuario tiene autorización para ejecutar una acción.
- **Caching**: Almacenar los resultados de funciones costosas para evitar recalcularlos.
- **Simplificación de APIs en librerías**: Abstraer lógica compleja, como veremos con el SDK de OpenAI.

## 2. Ejemplos Simples: Explorando `@classmethod` y `@staticmethod`

Antes de crear nuestros propios decoradores, es útil entender dos de los más comunes que vienen incorporados en Python y se usan en el contexto de las clases.

In [42]:
class Calculadora:
    # Método de instancia regular. Necesita una instancia (self) para operar.
    def sumar(self, a, b):
        return a + b

    # Un método estático no recibe la instancia (self) ni la clase (cls) como primer argumento.
    # Es esencialmente una función normal que vive dentro del 'namespace' de la clase.
    @staticmethod
    def info():
        print("Esta es una clase de calculadora simple.")

    # Un método de clase recibe la clase misma (cls) como primer argumento.
    # Es útil para crear 'factory methods', que construyen instancias de la clase de formas alternativas.
    @classmethod
    def crear_con_valor(cls, valor):
        # Aquí podríamos tener una lógica más compleja para crear la instancia
        print(f"Creando una instancia de {cls.__name__}...")
        return cls()

# Uso de los métodos:

# El método estático se puede llamar directamente desde la clase
Calculadora.info()

# El método de clase también se llama desde la clase
mi_calc = Calculadora.crear_con_valor(10)

# El método de instancia requiere un objeto creado
resultado = mi_calc.sumar(5, 3)
print(f"Resultado de la suma: {resultado}")

Esta es una clase de calculadora simple.
Creando una instancia de Calculadora...
Resultado de la suma: 8


## 3. Ejemplo Avanzado: Utilizando `@function_tool` del SDK de Agentes de OpenAI

Las librerías modernas utilizan decoradores para simplificar interacciones complejas. Un excelente ejemplo es el decorador `@function_tool` del SDK de Agentes de OpenAI.

Su propósito es tomar una función de Python estándar y convertirla en una "herramienta" que un modelo de lenguaje (como GPT-4) puede entender y decidir invocar. El decorador se encarga de generar automáticamente el esquema JSON que la API de OpenAI necesita para registrar la función, una tarea que de otro modo sería manual y propensa a errores.

In [43]:
# NOTA: Este código es para fines demostrativos.
# Necesitaría la librería 'openai' instalada y un cliente configurado para ejecutarse.

from agents import function_tool

# Definimos una función normal de Python con type hints y un docstring claro.
@function_tool
def obtener_clima(ciudad: str, unidad: str = "celsius"):
    """Obtiene el clima actual para una ciudad específica."""
    # En una aplicación real, aquí iría la lógica para llamar a una API de clima.
    if "bogota" in ciudad.lower():
        return f"El clima en Bogotá es de 15 grados {unidad}."
    return f"No se encontró información del clima para {ciudad}."

# El decorador @function_tool ha envuelto nuestra función.
# Internamente, ha generado un esquema JSON a partir de la firma de la función y su docstring.
# Este esquema es lo que un Agente de OpenAI usaría para saber cómo llamar a nuestra función.

print("La función 'obtener_clima' ha sido registrada como una herramienta.")
print("Un agente de IA ahora podría invocarla, por ejemplo: obtener_clima(ciudad='Bogota')")

La función 'obtener_clima' ha sido registrada como una herramienta.
Un agente de IA ahora podría invocarla, por ejemplo: obtener_clima(ciudad='Bogota')


## 4. Detrás de Escenas: Creando decoradores personalizados

Entender cómo funcionan los decoradores internamente es clave para dominarlos. La estructura básica es una función anidada:

1.  **Función Externa (El Decorador)**: Acepta la función a decorar como argumento.
2.  **Función Interna (El `wrapper` o envoltura)**: Aquí se añade la nueva funcionalidad. Esta función es la que finalmente ejecuta la función original.
3.  El decorador **devuelve la función `wrapper`**.

In [44]:
import time
import functools

def medir_tiempo(func):
    # @functools.wraps preserva los metadatos de la función original (nombre, docstring, etc.)
    # Es una buena práctica usarlo siempre.
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Lógica ANTES de llamar a la función original
        print(f"▶️ Ejecutando '{func.__name__}'...")
        inicio = time.time()
        
        # Llamada a la función original
        resultado = func(*args, **kwargs)
        
        # Lógica DESPUÉS de llamar a la función original
        fin = time.time()
        print(f"✅ '{func.__name__}' finalizó en {fin - inicio:.4f} segundos.")
        return resultado
    return wrapper

# Aplicamos nuestro decorador personalizado
@medir_tiempo
def proceso_largo(duracion):
    """Esta función simula un proceso que tarda un tiempo en completarse."""
    time.sleep(duracion)
    return "Proceso completado"

# Llamamos a la función decorada
proceso_largo(1)

# Verificamos que los metadatos se conservaron gracias a @functools.wraps
print(f"\nNombre de la función: {proceso_largo.__name__}")
print(f"Docstring: {proceso_largo.__doc__}")

▶️ Ejecutando 'proceso_largo'...
✅ 'proceso_largo' finalizó en 1.0004 segundos.

Nombre de la función: proceso_largo
Docstring: Esta función simula un proceso que tarda un tiempo en completarse.


## Conclusión

Los decoradores son una herramienta de metaprogramación que fomenta la escritura de código **DRY (Don't Repeat Yourself)**. Permiten separar las responsabilidades transversales (como logging o monitoreo) de la lógica de negocio principal, resultando en un código más limpio, modular y fácil de mantener.